This notebook is meant to dive into the constraint objects used in the optimization.

We currently have two classes of constraints to consider
1. Lower bounds as given by GPP fluxes
2. Lower and upper bounds as given by wanting to keep some scaling factors fixed (e.g., January)

We investigate1
1. January constraints
2. In a toy problem, the effect of encoding equality constraints as inequality constraints

In [1]:
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import pickle

from generate_opt_objects import A_b_generation

**PNC:/Users/mikestanley/anaconda3/envs/mc_sampling/lib/python3.8/site-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  pyproj could not be found, so IO/API coordinates cannot be converted to lat/lon; to fix, install pyproj or basemap (e.g., `pip install pyproj)`


In [2]:
# read in box constraints
with open('./data/carbon_flux/objects/scipy_bnds.pkl', 'rb') as f:
    box_constraints = pickle.load(f)

In [7]:
# generate objects WITH GPP fluxes
A, b = A_b_generation(
    box_constraint_fp='./data/carbon_flux/objects/scipy_bnds.pkl'
)
print(A.shape)
print(b.shape)

(11120, 26496)
(11120,)


# Isolate Jan

In this section, I test out using the `A_b_generation` function with a list of unity settings.

In [4]:
JAN_UP_IDX = 46 * 72

In [5]:
# create the unity indices array
unity_indices = np.arange(JAN_UP_IDX)
print(unity_indices)

[   0    1    2 ... 3309 3310 3311]


In [8]:
# generate the constraint objects
A_jan_unity, b_jan_unity = A_b_generation(
    box_constraint_fp='./data/carbon_flux/objects/scipy_bnds.pkl',
    unity_indices=unity_indices
)
print(A_jan_unity.shape)
print(b_jan_unity.shape)

(17744, 26496)
(17744,)


In [11]:
A_jan_unity[-2 * 3311, :]

array([ 0.,  0., -1., ...,  0.,  0.,  0.])

In [12]:
b_jan_unity[-2 * 3311]

-1.0

# Toy Problem -- replacing equality constraints with inequality constraints

Consider the problem
\begin{align}
\min \quad& h^T x \\
\text{subject to} \quad& x \geq 0 \\
\quad& c^T x = \mu.
\end{align}

This problem can be equivalently formulated as
\begin{align}
\min \quad& h^T x \\
\text{subject to} \quad& x \geq 0 \\
\quad& c^T x \leq \mu \\
\quad& c^T x \geq \mu.
\end{align}

Here, we investigate the outcome of that choice for a particular choice of $h$, $c$, and $\mu$.

In [9]:
# define h, c, and mu
h = np.array([1, -1])
c = np.array([1, 1])
mu  = 1

In [12]:
# equality version
x_eq = cp.Variable(2)
const_eq = [
    x_eq >= 0,
    c @ x_eq == mu
]
prob_eq = cp.Problem(
    objective=cp.Minimize(h @ x_eq),
    constraints=const_eq
)
prob_eq.solve(solver=cp.ECOS)

-0.9999999974513755

In [13]:
# inequality version
x_ineq = cp.Variable(2)
const_ineq = [
    x_ineq >= 0,
    c @ x_ineq <= mu,
    c @ x_ineq >= mu
]
prob_ineq = cp.Problem(
    objective=cp.Minimize(h @ x_ineq),
    constraints=const_ineq
)
prob_ineq.solve(solver=cp.ECOS)

-1.000000000129396

In [15]:
print(const_eq[0].dual_value)
print(const_ineq[0].dual_value)

[2.00000000e+00 3.80250169e-10]
[2.00000000e+00 9.99448909e-10]


In [16]:
print(const_eq[1].dual_value)
print(const_ineq[1].dual_value)

0.9999999974026244
2.1114540364456778


In [17]:
print(const_eq[1].dual_value)
print(const_ineq[2].dual_value)

0.9999999974026244
1.1114540362780154


In [19]:
x_ineq.value, x_eq.value

(array([9.91113036e-11, 1.00000000e+00]),
 array([1.27434439e-09, 9.99999999e-01]))

# Toy Problem -- Nocedal/Wright

We have the equality-constrained problem:
\begin{align}
\min \quad& x_1 + x_2 \\
\text{subject to} \quad& x_1^2 + x_2^2 - 2 = 0
\end{align}

and the inequality-constrained problem,
\begin{align}
\min \quad& x_1 + x_2 \\
\text{subject to} \quad& x_1^2 + x_2^2 - 2 \geq 0
\end{align}

In [4]:
h_nw = np.array([1, 1])

In [7]:
# equality version
x_eq = cp.Variable(2)
const_eq = [
    cp.quad_form(x_eq, np.identity(2)) - 2 == 0
]
prob_eq = cp.Problem(
    objective=cp.Minimize(h_nw @ x_eq),
    constraints=const_eq
)
prob_eq.solve(solver=cp.ECOS)

DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
QuadForm(var11, [[1.00 0.00]
 [0.00 1.00]]) + -2.0 == 0.0 , because the following subexpressions are not:
|--  QuadForm(var11, [[1.00 0.00]
 [0.00 1.00]]) + -2.0 == 0.0

In [8]:
# equality version
x_ineq = cp.Variable(2)
const_ineq = [
    cp.quad_form(x_ineq, np.identity(2)) - 2 >= 0
]
prob_ineq = cp.Problem(
    objective=cp.Minimize(h_nw @ x_ineq),
    constraints=const_ineq
)
prob_ineq.solve(solver=cp.ECOS)

DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
0.0 <= QuadForm(var20, [[1.00 0.00]
 [0.00 1.00]]) + -2.0 , because the following subexpressions are not:
|--  0.0 <= QuadForm(var20, [[1.00 0.00]
 [0.00 1.00]]) + -2.0